How can I get a time-series (in order visual) of every df used in this dq pipeline. I want to see how it starts and how it branches and transforms through the pipeline. I also want to see when and where variables are created and what they are connected to visually.. Can I create something that makes it look like a plumbers pipe network?


Yes, you 100% can do this – and it’s a *perfect* use-case for a little lineage registry + graph.

Think of it in two layers:

1. **A timeline / lineage table** of every “version” of `df` you create.
2. **A DAG / graph** that shows how those versions flow and branch – your plumber’s pipe network.

Below is a concrete pattern you can drop into your project.

---

## 1. Add a tiny lineage registry (one-time helper)

Put this in a helpers module, e.g. `src/telco_churn/lineage.py` (so your 2.x notebook can still be “no-def”):

```python
# src/telco_churn/lineage.py
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import hashlib
import time

@dataclass
class DFNode:
    node_id: str           # unique id for the node (e.g. "step_2.3.1_df_profiled")
    name: str              # friendly name (e.g. "df_profiled")
    step: str              # section/step id (e.g. "2.3.1")
    description: str       # what this step did
    parents: List[str]     # list of parent node_ids
    n_rows: int
    n_cols: int
    cols_added: List[str]
    cols_removed: List[str]
    timestamp: float       # time when registered (for ordering)
    sample_hash: str       # hash of a small sample for “shape-of-data” fingerprint

class DFLineageRegistry:
    def __init__(self):
        self.nodes: Dict[str, DFNode] = {}
        self.last_node_for_dfname: Dict[str, str] = {}  # df_name -> node_id

    def _hash_sample(self, df: pd.DataFrame, n: int = 10) -> str:
        if df.empty:
            return "empty"
        sample = df.head(n).to_json(orient="split", default_handler=str)
        return hashlib.md5(sample.encode("utf-8")).hexdigest()

    def register(
        self,
        df: pd.DataFrame,
        df_name: str,
        step: str,
        description: str = "",
        parents: Optional[List[str]] = None,
    ) -> str:
        """Register a new DF state and return its node_id."""
        if parents is None:
            parents = []

        # infer parents from last node of this df_name if none provided
        if not parents and df_name in self.last_node_for_dfname:
            parents = [self.last_node_for_dfname[df_name]]

        node_id = f"{step}_{df_name}"

        # column diffs vs first parent (if any)
        cols = list(df.columns)
        cols_added, cols_removed = [], []
        if parents:
            parent_node = self.nodes[parents[0]]
            parent_cols = set(parent_node.cols_added) | set(parent_node.cols_removed) \
                          if False else set()  # we’ll recompute from df sample
            # simpler: use parent snapshot stored in registry? (future enhancement)
        # for now: we’ll compute added/removed by storing the parent’s column set separately:
        # to keep it simple, we won’t use that trick here – just leave cols_added/removed',
        # or fill them in manually later if you want.

        n_rows = int(df.shape[0])
        n_cols = int(df.shape[1])

        node = DFNode(
            node_id=node_id,
            name=df_name,
            step=step,
            description=description,
            parents=parents,
            n_rows=n_rows,
            n_cols=n_cols,
            cols_added=cols_added,
            cols_removed=cols_removed,
            timestamp=time.time(),
            sample_hash=self._hash_sample(df),
        )

        self.nodes[node_id] = node
        self.last_node_for_dfname[df_name] = node_id
        return node_id

    def to_dataframe(self) -> pd.DataFrame:
        return pd.DataFrame([asdict(n) for n in self.nodes.values()])
```

Then, in your notebook:

```python
from telco_churn.lineage import DFLineageRegistry

df_lineage = DFLineageRegistry()
```

---

## 2. Instrument your pipeline steps

Now, whenever you have a meaningful “version” of a dataframe, register it.

### Example: early pipeline

```python
# After you load raw data
df = pd.read_csv(raw_path)
node_raw = df_lineage.register(
    df=df,
    df_name="df",
    step="2.0.0",
    description="Loaded raw Telco CSV"
)

# After basic type coercions / schema normalization (2.1.x)
# df = your coerced version
node_21 = df_lineage.register(
    df=df,
    df_name="df",
    step="2.1.5",
    description="Post core numeric/categorical type alignment"
)

# After profiling / quality diagnostics (2.3 final df used for diagnostics)
node_23 = df_lineage.register(
    df=df,
    df_name="df",
    step="2.3.14",
    description="Post numeric/categorical diagnostics – ready for cleaning"
)
```

### Example: cleaning branch (using your 2.6 convention)

If you’re using `df_before_clean` and `df_clean`:

```python
# 2.6.0 – before & working copy
df_before_clean = df.copy(deep=True)
df_clean = df.copy(deep=True)

node_before_clean = df_lineage.register(
    df=df_before_clean,
    df_name="df_before_clean",
    step="2.6.0",
    description="Snapshot before controlled cleaning"
)

node_clean_260 = df_lineage.register(
    df=df_clean,
    df_name="df_clean",
    step="2.6.0",
    description="Initial working copy for cleaning"
)

# After a cleaning step, e.g. 2.6.3 missing value imputation
# df_clean[...] = ...
node_clean_263 = df_lineage.register(
    df=df_clean,
    df_name="df_clean",
    step="2.6.3",
    description="After missing value imputation",
)
```

If you ever **branch** (e.g. create a temporary df for a scenario):

```python
df_outlier_clip = df_clean[df_clean["MonthlyCharges"] < 300].copy()

node_outlier_clip = df_lineage.register(
    df=df_outlier_clip,
    df_name="df_outlier_clip",
    step="2.6.4",
    description="Outlier-clipped copy for analysis",
    parents=[node_clean_263],  # explicitly set parent to show branch
)
```

Later if you merge back:

```python
df_clean = df_outlier_clip  # or some merge logic
node_clean_265 = df_lineage.register(
    df=df_clean,
    df_name="df_clean",
    step="2.6.5",
    description="After applying outlier decisions back to df_clean",
    parents=[node_outlier_clip],
)
```

Now you’re accumulating a **time-ordered lineage of every dataframe** used in the pipeline.

---

## 3. See the time-series / evolution (table + simple plots)

At any point:

```python
lineage_df = df_lineage.to_dataframe().sort_values("timestamp")
display(lineage_df[["step", "name", "n_rows", "n_cols", "description"]])
```

You can quickly plot, say, **row count over time**:

```python
import matplotlib.pyplot as plt

lineage_df = df_lineage.to_dataframe().sort_values("timestamp")

plt.figure()
plt.plot(lineage_df["step"], lineage_df["n_rows"], marker="o")
plt.xticks(rotation=90)
plt.ylabel("n_rows")
plt.title("Row count over pipeline steps")
plt.tight_layout()
plt.show()
```

That gives you the **time-series** feel: how the dataset size changes as it flows through the pipeline.

---

## 4. Build the plumber-pipe graph (DAG)

For the “pipe network” visual, Graphviz is really nice.

```python
from graphviz import Digraph

def build_dq_graph(registry: DFLineageRegistry) -> Digraph:
    dot = Digraph(comment="Section 2 Data Quality Pipeline")
    dot.attr(rankdir="LR")  # left-to-right like a pipe

    # nodes
    for node in registry.nodes.values():
        label = f"{node.step}\\n{node.name}\\nrows={node.n_rows}, cols={node.n_cols}"
        dot.node(
            node.node_id,
            label=label,
            shape="box",
            style="rounded,filled",
        )

    # edges
    for node in registry.nodes.values():
        for parent_id in node.parents:
            dot.edge(parent_id, node.node_id)

    return dot

dq_graph = build_dq_graph(df_lineage)
dq_graph
```

In Jupyter this will render as a **DAG**:

* **Boxes** are dataframe states.
* **Arrows** are transformations.
* The left-to-right flow gives the “pipes” vibe.

You can tweak “plumber aesthetics” further:

```python
dot.attr("node", shape="box", style="rounded,filled", penwidth="1.5")
dot.attr("edge", penwidth="1.5")
```

You can also color nodes by **phase**:

* 2.0–2.2 = ingestion/sanity (blue)
* 2.3–2.5 = diagnostics (orange)
* 2.6 = cleaning/apply (green)

(Just parse `node.step` and set `fillcolor` based on prefixes.)

---

## 5. Tracking *where variables/columns are created*

If you want to see **which step created which columns**, two options:

### Option A – cheap & good enough

When you know you’re *adding* columns in a step, you can explicitly annotate in the description:

```python
node_2513 = df_lineage.register(
    df=df,
    df_name="df",
    step="2.5.13",
    description="Added flags: has_fiber, high_charges_flag"
)
```

Then you can filter `lineage_df` for descriptions containing `"Added"` or whatever pattern you choose.

### Option B – automated column diff

If you’re okay adding a bit more machinery:

* Store, inside `DFNode`, a `columns` field (list of column names).

* When registering a new node, if there’s a parent, compute:

  ```python
  cols_added = sorted(list(set(current_cols) - set(parent_cols)))
  cols_removed = sorted(list(set(parent_cols) - set(current_cols)))
  ```

* Save `cols_added` / `cols_removed` in the node.

Then you can do:

```python
lineage_df = df_lineage.to_dataframe().explode("cols_added")
new_cols = lineage_df[~lineage_df["cols_added"].isna()][["step", "cols_added", "description"]]
display(new_cols)
```

That becomes a **“column origin map”**: each column and the step where it first appeared.

We can also, if you want later, build a **bipartite graph** (nodes = dataframe snapshots + column nodes) so it *really* looks like a complex pipe manifold – but the simple DAG with annotated labels is usually enough and easier to read.

---

If you’d like, I can next:

* 💡💡 Sketch a concrete `DFLineageRegistry` v2 that **automatically** tracks `columns`, `cols_added`, `cols_removed` correctly, and
* 💡💡 Show how to layer this into your existing **Section 2** naming (2.0.0, 2.1.3, 2.3.14, 2.6.10, etc.) with example calls right where you already have `print("2.5.3 …")` etc.


Can you create a version I don't have to put in the notebook but abstracts any "df" variable throughout the entire thing?

Totally love this idea – “plumber’s pipe network” is exactly what data lineage *should* feel like. 🧑‍🔧🪠

You *can’t* magically track every dataframe in a notebook with **zero** hooks, but we can get **very close**:

* Put **all the logic in a separate module** (`lineage_auto.py`).
* In the notebook, you just sprinkle **one small line per major step**, and it will:

  * auto-discover any `df*` variable in `globals()` (e.g. `df`, `df_clean`, `df_before_clean`, `df_num_profile`, etc.),
  * register them in a lineage registry,
  * compute column diffs vs the previous version,
  * later render:

    * a **time-series table**, and
    * a **Graphviz DAG** that looks like a pipe network.

Below is that module + how to use it.

---

## 1. Drop this into `telco_churn/lineage_auto.py`

```python
# telco_churn/lineage_auto.py

from __future__ import annotations

from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Iterable, Callable
import pandas as pd
import numpy as np  # noqa: F401  # might be handy later
import hashlib
import time

try:
    from graphviz import Digraph
except ImportError:
    Digraph = None  # graceful fallback


# -------------------------------------------------------------------
# Core data structure: one node = one snapshot of a df-like variable
# -------------------------------------------------------------------
@dataclass
class DFNode:
    node_id: str           # unique id: f"{step}:{df_name}"
    df_name: str           # variable name: "df", "df_clean", "df_before_clean", etc.
    step: str              # pipeline step: "2.0.0", "2.3.14", "2.6.10", ...
    description: str       # human text of what we did
    parents: List[str]     # node_ids of parents
    n_rows: int
    n_cols: int
    columns: List[str]
    cols_added: List[str]
    cols_removed: List[str]
    timestamp: float       # time.time()
    sample_hash: str       # md5 of small JSON sample for "shape" fingerprint


class DFLineageRegistry:
    """
    Registry that tracks df-like variables across steps.
    You don't touch this directly much; you'll mostly call register_step().
    """

    def __init__(self):
        self.nodes: Dict[str, DFNode] = {}
        self.last_node_for_dfname: Dict[str, str] = {}
        self._columns_for_node: Dict[str, List[str]] = {}

    # ------------- internals ------------- #

    def _hash_sample(self, df: pd.DataFrame, n: int = 10) -> str:
        if df is None or getattr(df, "empty", False):
            return "empty"
        try:
            sample = df.head(n).to_json(orient="split", default_handler=str)
        except Exception:
            sample = "unhashable"
        return hashlib.md5(sample.encode("utf-8")).hexdigest()

    def _compute_col_diff(
        self,
        current_cols: Iterable[str],
        parent_node_id: Optional[str],
    ) -> (List[str], List[str]):
        current_cols = list(current_cols)
        if not parent_node_id or parent_node_id not in self._columns_for_node:
            # No parent info: can't diff; treat everything as "existing" not "added"
            return [], []
        parent_cols = set(self._columns_for_node[parent_node_id])
        curr_cols_set = set(current_cols)
        cols_added = sorted(list(curr_cols_set - parent_cols))
        cols_removed = sorted(list(parent_cols - curr_cols_set))
        return cols_added, cols_removed

    # ------------- public methods ------------- #

    def register_df(
        self,
        df: pd.DataFrame,
        df_name: str,
        step: str,
        description: str = "",
        parents: Optional[List[str]] = None,
    ) -> str:
        """
        Register a single df snapshot.

        Returns
        -------
        node_id : str
            The ID of the created node (e.g. "2.3.14:df_clean").
        """
        if parents is None:
            parents = []

        # If parents not explicitly set, infer from last node of same df_name
        if not parents and df_name in self.last_node_for_dfname:
            parents = [self.last_node_for_dfname[df_name]]

        node_id = f"{step}:{df_name}"

        # Basic shape
        n_rows = int(df.shape[0])
        n_cols = int(df.shape[1])
        cols = list(map(str, df.columns))

        # Column diff vs first parent (if any)
        parent_id = parents[0] if parents else None
        cols_added, cols_removed = self._compute_col_diff(cols, parent_id)

        node = DFNode(
            node_id=node_id,
            df_name=df_name,
            step=step,
            description=description,
            parents=parents,
            n_rows=n_rows,
            n_cols=n_cols,
            columns=cols,
            cols_added=cols_added,
            cols_removed=cols_removed,
            timestamp=time.time(),
            sample_hash=self._hash_sample(df),
        )

        self.nodes[node_id] = node
        self.last_node_for_dfname[df_name] = node_id
        self._columns_for_node[node_id] = cols
        return node_id

    def register_step(
        self,
        namespace: Dict[str, object],
        step: str,
        description: str = "",
        name_filter: Optional[Callable[[str], bool]] = None,
    ) -> List[str]:
        """
        Auto-register all df-like variables in a given namespace
        (e.g. globals()) for a given pipeline step.

        Parameters
        ----------
        namespace : dict
            Usually globals() from your notebook cell.
        step : str
            Step identifier like "2.3.14" or "2.6.10".
        description : str
            Human text for this step (what it represents).
        name_filter : callable or None
            Function name -> bool. If None, defaults to variables whose names
            start with "df" (df, df_clean, df_before_clean, df_num_profile, etc.).

        Returns
        -------
        node_ids : list of str
            Node IDs registered for this step.
        """
        if name_filter is None:
            def name_filter(n: str) -> bool:
                return n.startswith("df")

        node_ids: List[str] = []
        for name, value in namespace.items():
            if not name_filter(name):
                continue
            # Only track pandas DataFrame objects
            if isinstance(value, pd.DataFrame):
                node_id = self.register_df(
                    df=value,
                    df_name=name,
                    step=step,
                    description=description,
                )
                node_ids.append(node_id)
        return node_ids

    # ----------- export helpers ----------- #

    def to_dataframe(self) -> pd.DataFrame:
        if not self.nodes:
            return pd.DataFrame(columns=[
                "node_id", "df_name", "step", "description",
                "parents", "n_rows", "n_cols",
                "columns", "cols_added", "cols_removed",
                "timestamp", "sample_hash",
            ])
        return pd.DataFrame([asdict(n) for n in self.nodes.values()])

    def build_graph(self, rankdir: str = "LR") -> "Digraph":
        """
        Build a Graphviz Digraph representing the lineage graph.

        rankdir:
            "LR" = left-to-right (pipe-like),
            "TB" = top-to-bottom, etc.
        """
        if Digraph is None:
            raise ImportError(
                "graphviz is not installed. `pip install graphviz` "
                "and make sure Graphviz binaries are available."
            )

        dot = Digraph(comment="DQ Pipeline Lineage")
        dot.attr(rankdir=rankdir)
        dot.attr("node", shape="box", style="rounded,filled", penwidth="1.5")
        dot.attr("edge", penwidth="1.5")

        # Node colors by phase for plumber-y vibe
        def color_for_step(step: str) -> str:
            if step.startswith("2.0") or step.startswith("2.1") or step.startswith("2.2"):
                return "#cfe2ff"  # ingestion/schema
            if step.startswith("2.3") or step.startswith("2.4") or step.startswith("2.5"):
                return "#ffe5b4"  # diagnostics
            if step.startswith("2.6"):
                return "#d3f9d8"  # cleaning/apply
            return "#e2e3e5"      # default/other

        for node in self.nodes.values():
            label = (
                f"{node.step}\\n"
                f"{node.df_name}\\n"
                f"rows={node.n_rows}, cols={node.n_cols}"
            )
            dot.node(
                node.node_id,
                label=label,
                fillcolor=color_for_step(node.step),
            )

        for node in self.nodes.values():
            for parent_id in node.parents:
                if parent_id in self.nodes:
                    dot.edge(parent_id, node.node_id)

        return dot


# Global singleton you can import & use everywhere
LINEAGE = DFLineageRegistry()
```

That’s it – this file lives in your project, *not* in the notebook.

---

## 2. Minimal usage in the notebook

You only need **1–2 tiny lines per major step**, no heavy code.

At the top of the Section 2 notebook:

```python
from telco_churn.lineage_auto import LINEAGE
```

Then, at natural “checkpoints” (end of 2.0.0, 2.1.x, 2.3.14, 2.6.0, 2.6.10, etc.), add:

```python
# After 2.0.0 bootstrap / load
LINEAGE.register_step(
    globals(),
    step="2.0.0",
    description="Bootstrap & load raw Telco data"
)

# After 2.1.x schema normalization
LINEAGE.register_step(
    globals(),
    step="2.1.5",
    description="Post core numeric/categorical type alignment"
)

# After 2.3.14 numeric/categorical diagnostics
LINEAGE.register_step(
    globals(),
    step="2.3.14",
    description="Diagnostics complete – ready for controlled cleaning"
)

# 2.6.0 – right after you create df_before_clean / df_clean
LINEAGE.register_step(
    globals(),
    step="2.6.0",
    description="Snapshot before cleaning & initial df_clean"
)

# After a big cleaning step (say, missing value imputation)
LINEAGE.register_step(
    globals(),
    step="2.6.3",
    description="After missing-value imputation rules"
)

# After full 2.6.x completed
LINEAGE.register_step(
    globals(),
    step="2.6.12",
    description="Final cleaned df + audit metadata"
)
```

Because `register_step`:

* looks at `globals()`,
* finds any variable whose name **starts with `"df"`**,
* and is a `pd.DataFrame`,

you don’t have to spell out `df`, `df_clean`, `df_before_clean`, `df_cat_profile`, etc. It just scoops them all and wires parent relationships based on previous registrations of the same name.

So it **abstracts any `df*` variable** without cluttering your notebook with lineage internals.

---

## 3. Time-series / evolution view

At the end (or whenever you want a snapshot):

```python
lineage_df = LINEAGE.to_dataframe().sort_values("timestamp")

# High-level evolution:
display(
    lineage_df[
        ["step", "df_name", "n_rows", "n_cols", "cols_added", "cols_removed", "description"]
    ]
)
```

If you want a quick time-series for `df_clean` only:

```python
df_clean_hist = (
    lineage_df[lineage_df["df_name"] == "df_clean"]
    .sort_values("timestamp")
)

display(df_clean_hist[["step", "n_rows", "n_cols", "cols_added", "cols_removed"]])
```

You’ll see how `df_clean` starts and changes over 2.6.x.

---

## 4. Plumber-pipe DAG visual

If you have `graphviz` installed:

```python
from telco_churn.lineage_auto import LINEAGE

dq_graph = LINEAGE.build_graph(rankdir="LR")  # LR = left→right pipes
dq_graph
```

That gives you a **graph** where:

* Each box is a df snapshot (`2.3.14\ndf`, `2.6.0\ndf_before_clean`, `2.6.3\ndf_clean`, etc.).
* Arrows show flow.
* Colors indicate phase:

  * blue-ish = 2.0–2.2 (ingestion/schema),
  * orange-ish = 2.3–2.5 (diagnostics),
  * green-ish = 2.6 (cleaning).

Which is very much the “copper pipes running through your wall” vibe.

If you want to go even more plumber-core later, we can:

* 💡💡 Add edge labels for “what happened here?” (`Missing value rules`, `Outlier winsorization`, etc.).
* 💡💡 Add a column-level bipartite option (nodes for columns + nodes for df snapshots), so you can trace an individual column through the pipe maze.

---

If you paste this module and the small `LINEAGE.register_step(globals(), ...)` calls into your flow, your pipeline will start *documenting itself* as it runs, without stuffing a ton of logic into the notebook itself.
